In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import glm_model as gm
from glm_utils import *
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#finally get rid of the ugly ass allen code. We import the pickled design 
#matrix stimulus, which is PCA projection of the images, and the downsampled
#traces. 

import pickle

pkl_file = open('./boc/formatted/LMcelldata')
data = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/LMstimdata')
stim = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/runspeed')
run_speed = pickle.load(pkl_file)
pkl_file.close()

In [3]:
#get the number of features, and the number of training examples. 
l, n =  stim.shape

In [10]:
from statsmodels.tsa.tsatools import lagmat
from sklearn.cross_validation import train_test_split
from scipy.stats import linregress
from itertools import product
import pyprind

import sys

n_pcs = range(1, 110, 5)
non_lin = [0]

alpha = 0
nlag = 2

#empty dictionary of the best model we've fit for each cell
best_model = {k: (-np.inf, None) for k in data.keys()}

#make a dictionary of dictionaries to store the results
scores_dict = {k: {} for k in data.keys()}

bar = pyprind.ProgBar(len(n_pcs)*len(non_lin))

for (l, n) in product(non_lin, n_pcs):
    bar.update()
    
    #lagged_stim = lagmat(stim[:, 0:n], maxlag = nlag, trim = None)[1:-1]       
    stim_array = np.concatenate((stim[:, 0:n], run_speed[:, np.newaxis]), axis = 1)
    
    
    for c in data.keys():
        X_train, X_test, y_train, y_test = train_test_split(stim_array, np.array(data[c]))     
        weight_init = np.linalg.pinv(X_train).dot(y_train)[:, np.newaxis]
       
        if l == 0:
            nonlin = tf.exp
        if l == 1:
            nonlin = tf.nn.sigmoid
        if l == 2:
            nonlin = tf.nn.relu

        
        model = gm.exponential_GLM(weight_init, non_lin = nonlin, verbose = False, alpha = 2, reg = 'l2', 
                                lr = 1e-4, train_params = False, scale_init = 1, bias_init = 0)
        
        loss, cv = model.fit(X_train, y_train[:, np.newaxis], X_test, y_test[:, np.newaxis], 
                             max_iters = 200) 
        
        
        w, o, s  = model.get_params()

    
        _, _, r2_test,_, _ = linregress(np.squeeze(model.predict_trace(X_test)), np.squeeze(y_test))
        _, _, r2_train,_, _ = linregress(np.squeeze(model.predict_trace(X_train)), np.squeeze(y_train))

        scores_dict[c][(l, n)] = (r2_test, r2_train) 

        if r2_test > best_model[c][0]:
            best_model[c] = (r2_test, (l, n))
            print "new best model for cell ", c, ". r2 test: ", r2_test


0%                  100%
[#                     ] | ETA: 00:00:00

new best model for cell  517442753 . r2 test:  -0.0032444108793
new best model for cell  517444738 . r2 test:  0.136609957688
new best model for cell  517443331 . r2 test:  -0.0410437017771
new best model for cell  517442436 . r2 test:  -0.145465451173
new best model for cell  517445575 . r2 test:  -0.341249856758
new best model for cell  517444426 . r2 test:  -0.0882227873337
new best model for cell  517442471 . r2 test:  0.00888691472904
new best model for cell  517443181 . r2 test:  -0.0002619712879
new best model for cell  517443087 . r2 test:  -0.143584330928
new best model for cell  517446833 . r2 test:  0.180404086809
new best model for cell  517443827 . r2 test:  -0.0451576192325
new best model for cell  517443924 . r2 test:  -0.102103213929
new best model for cell  517443957 . r2 test:  0.0348906321693
new best model for cell  517442722 . r2 test:  -0.0584865250145
new best model for cell  517446488 . r2 test:  -0.171758777013
new best model for cell  517442675 . r2 test:  0.0

[###                   ] | ETA: 01:34:44

 517443571 . r2 test:  0.0349838293376
new best model for cell  517442436 . r2 test:  -0.0367410964423
new best model for cell  517445575 . r2 test:  -0.00889788980436
new best model for cell  517444426 . r2 test:  0.0302751394295
new best model for cell  517443087 . r2 test:  0.0225310762474
new best model for cell  517446833 . r2 test:  0.23258456535
new best model for cell  517443827 . r2 test:  0.00540624358916
new best model for cell  517443924 . r2 test:  0.0356070348689
new best model for cell  517442722 . r2 test:  0.00722710646601
new best model for cell  517446554 . r2 test:  0.0979729836392
new best model for cell  517444175 . r2 test:  -0.0131772987335
new best model for cell  517447069 . r2 test:  0.00721363536958
new best model for cell  517443331 . r2 test:  -0.0150850018353
new best model for cell  517442436 . r2 test:  -0.0299779243697
new best model for cell  517442471 . r2 test:  0.023624403498
new best model for cell  517442722 . r2 test:  0.0620534115625
new best m

[#####                 ] | ETA: 01:41:28

...

[####################  ] | ETA: 00:33:45

 517446488 . r2 test:  0.0420696375904
new best model for cell 

[######################] | ETA: 00:00:00

 517442675 . r2 test:  0.0583723465233



Total time elapsed: 06:23:53


In [15]:
for c in data.keys():
    
    if best_model[c][1] != None:
        (v, (l, n)) = best_model[c]
    
        print 'explained var: ', round(v, 3), 'non_lin:', l, 'n_components', n
        
    print best_model[c]

explained var:  0.205 non_lin: 0 n_components 1
(0.20471333787080059, (0, 1))
explained var:  0.067 non_lin: 0 n_components 1
(0.067399138643032402, (0, 1))
explained var:  -0.012 non_lin: 0 n_components 61
(-0.01225000620757246, (0, 61))
explained var:  0.321 non_lin: 0 n_components 1
(0.32066256818647759, (0, 1))
explained var:  0.243 non_lin: 0 n_components 21
(0.24282624633409033, (0, 21))
explained var:  0.174 non_lin: 0 n_components 1
(0.17381542853821436, (0, 1))
explained var:  0.047 non_lin: 0 n_components 1
(0.046700509571936236, (0, 1))
explained var:  0.012 non_lin: 0 n_components 81
(0.011955503314439762, (0, 81))
explained var:  0.112 non_lin: 0 n_components 1
(0.11185500602632083, (0, 1))
explained var:  0.263 non_lin: 0 n_components 21
(0.26327412441769821, (0, 21))
explained var:  0.005 non_lin: 0 n_components 61
(0.004968124413065044, (0, 61))
explained var:  0.182 non_lin: 0 n_components 1
(0.18216877669256215, (0, 1))
explained var:  0.118 non_lin: 0 n_components 1


In [ ]:
pkl_file = open('./boc/formatted/LNE_rs_exp', 'wb')
pickle.dump((best_model, scores_dict), pkl_file)
pkl_file.close()